# Do Photometry!

In [ ]:
# python 2/3 compatibility
from __future__ import print_function

# numerical python
import numpy as np

# file management tools
import glob
import os

# good module for timing tests
import time

# plotting stuff
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

# ability to read/write fits files
from astropy.io import fits

# fancy image combination technique
from astropy.stats import sigma_clipping

# median absolute deviation: for photometry
from astropy.stats import mad_std

# photometric utilities
from photutils import DAOStarFinder,aperture_photometry, CircularAperture, CircularAnnulus

# periodograms
from astropy.stats import LombScargle





In [ ]:
source = 'Taurus1'

datadir='/Volumes/A341/'+source+'/'

filters = ['V','R','I','Ha','Haoff']

#filters = ['R']



In [ ]:
aprad=10.  # aperture radius
skybuff=4.  # sky annulus inner radius
skywidth=8.  # sky annulus outer radius




In [ ]:
# sensitivities for star finding
nsigma=5. # detection threshold in sigma
FWHM=7. # pixels


### use a deep image to find the sources



In [ ]:

imname = glob.glob(datadir+'*R*')[0]

imname = '/Volumes/AST337/reduced/deep_Taurus1_R.fits'

im,hdr=fits.getdata(imname, header=True)
bkg_sigma = mad_std(im)
daofind = DAOStarFinder(fwhm=FWHM, threshold=nsigma*bkg_sigma, exclude_border=True)

sources = daofind(im)



In [ ]:
print(len(sources))

### use a plate-solved image to auto-match stars

In [ ]:
from HDI_io import construct_astrometry

In [ ]:
# check the coordinate options
hdulist = fits.open('/Volumes/A341/solved'+source+'.fits')


hdr_wcs = hdulist[0].header

world = construct_astrometry(hdr_wcs)



In [ ]:
Luhman = np.genfromtxt('Luhman16.txt',\
                        dtype={'names': ("RA","Dec","Flag","RAJ2000","DEJ2000",\
                                         "Name","Types","Date","Type","AJ","a",\
                                         "b","c","2Mass"),\
                             'formats': ('f16','f16','S20','S20','S20',\
                                        'S20','S20','S20','S20','f8','S20',\
                                        'S20','S20','S20')},\
                 skip_header=63,delimiter=';')




#### I also used a GAIA sourcelist to do cross matching

In [ ]:
GAIA = np.genfromtxt('/Users/mpetersen/Desktop/taurus1-result.csv',\
                        dtype={'names': ("source_id","ra","ra_error","dec","dec_error",\
                                         "parallax","parallax_error","phot_g_mean_mag","bp_rp",\
                                         "radial_velocity","radial_velocity_error","phot_variable_flag",\
                                         "teff_val","a_g_val"),\
                             'formats': ('f16','f16','f16','f16','f16',\
                                        'f16','f16','f16','f16',\
                                         'f16','f16','S20',\
                                        'f16','f16')},\
                 skip_header=1,delimiter=',')




In [ ]:
# place limits on stars that are at roughly the taurus distance

parall = 1000./GAIA['parallax'][GAIA['parallax'].argsort()]

gparall = np.where( (parall < 1000) & (parall > 50))
#plt.plot(parall[gparall],drawstyle='steps-mid')


In [ ]:
taura,taudec = world.all_pix2world(xpos,ypos,0)

In [ ]:

# one arcsecond matching
tol = 1./3600.
ngood=0

f = open(source+'_gaia.reg','w')



for src in range(0,len(GAIA['ra'])):

    dist = ( (np.cos(GAIA['dec'][src]*np.pi/180.)*(GAIA['ra'][src] - taura))**2.0 + \
                (GAIA['dec'][src] - taudec)**2.0 )**0.5
        
        
        # record the LMT values for later comparison
        #min_distribution[indx,2] = list1['Pointy'][indx]
        #min_distribution[indx,3] = list1['Blobby'][indx]

        
    x = np.where(dist==np.min(dist))[0][0]
    if np.min(dist) <= tol:
        ngood += 1
        
        if (1000./GAIA['parallax'][src] < 160) & (1000./GAIA['parallax'][src] > 120):
            f.write('circle '+str(xpos[x])+' '+str(ypos[x])+' '+str(int(aprad))+' # width=4 color=blue\n') # or whatever you want the radius to be!
            print(x)
        #else:
            #f.write('circle '+str(xpos[x])+' '+str(ypos[x])+' '+str(int(aprad))+' # width=4 color=red\n') # or whatever you want the radius to be!
        
        
print(ngood)
      
f.close()

### use daophot values to determine which sources have trouble with nebulosity

In [ ]:
plt.scatter(sources['sharpness'],sources['roundness1'],color='black',s=1.)
plt.scatter(sources['sharpness'],sources['roundness2'],color='red',s=1.)

In [ ]:
# apply sharpness and roundness cuts (somewhat ad hoc)
w = np.where( (sources['sharpness']>0.3) & (sources['sharpness']<0.9)\
             &(sources['roundness1']>-0.5) & (sources['roundness1']<0.5)\
             &(sources['roundness2']>-0.5) & (sources['roundness2']<0.5))

xpos = np.array(sources['xcentroid'])
ypos = np.array(sources['ycentroid'])

#xpos = xpos[w]
#ypos = ypos[w]

print(len(xpos))

nstars = len(xpos)
#positions = (sources['xcentroid'], sources['ycentroid'])
positions = (xpos,ypos)



### match the cleaned list to Luhman's Taurus catalog

In [ ]:

f = open(source+'taurussources.reg','w')


for star in range(0,len(Luhman["RA"])):
    try:
        x,y = world.all_world2pix(np.array(Luhman["RA"][star]).astype(np.float64),np.array(Luhman["Dec"][star]).astype(np.float64),0)



        #lon, lat = w.wcs_world2pix(Luhman["RA"][star],Luhman["Dec"][star])
        if ((x >= 0) & (x<hdulist[0].data.shape[0])\
            &(y >= 0) & (y<hdulist[0].data.shape[1])):
            
            
            snum = np.where( (np.abs(x - xpos) < 10.) &\
                          (np.abs(y - ypos) < 10.))[0]
            
            print(Luhman["Name"][star],np.round(x,0),np.round(y,0),snum)

            f.write('circle '+str(np.round(x,0))+' '+str(np.round(y,0))+' '+str(int(aprad))+' # width=4 color=red\n') # or whatever you want the radius to be!



    except:
        pass



f.close()




Print the sourcelist to RA-Dec coordinates

In [ ]:

f = open(source+'sources.txt','w')
for i in range(0,nstars):
    xout,yout = world.all_pix2world(xpos[i],ypos[i],0)

    f.write(str(i)+' '+str(xout)+' '+str(yout)+'\n')


f.close()

Also print in pixel coordinates for ds9

In [ ]:

f = open(source+'sources.reg','w')
for i in range(0,nstars):
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(int(aprad))+' # width=4 color=red\n') # or whatever you want the radius to be!
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(int(aprad+skybuff))+' # width=4 color=blue\n') # or whatever you want the radius to be!
    f.write('circle '+str(xpos[i])+' '+str(ypos[i])+' '+str(int(aprad+skybuff+skywidth))+' # width=4 color=blue\n') # or whatever you want the radius to be!



f.close()

## Run Photometry on Everything

In [ ]:
# make the master list of apertures

apertures = CircularAperture(positions, r=aprad)
annulus_apertures = CircularAnnulus(positions, r_in=aprad+skybuff, r_out=aprad+skybuff+skywidth)
apers = [apertures, annulus_apertures]

area_of_ap = apertures.area()
area_of_background = annulus_apertures.area()



In [ ]:
# this defines a box for noise properties. 
#it's best if it's empty...but won't be too thrown by faint stars.

boxsize = 200
xboxcorner = 2500
yboxcorner = 3000

GAIN = 1.3



Get ready to wait...

In [ ]:

Photometry = {}
ePhotometry = {}
Times = {}


for filtername in filters:
    
    # need the trailing '_' to avoid Ha/Haoff confusion
    imglist = glob.glob(datadir+'*'+filtername+'_*.fits')

    nimages = len(imglist)
    
    print('Found {} images'.format(nimages))
    
    # initialize the array
    Photometry[filtername] = np.zeros([nimages,nstars])
    ePhotometry[filtername] = np.zeros([nimages,nstars])
    Times[filtername] = np.zeros(nimages)



    for imgnum in range(0,nimages):
        
        print(imglist[imgnum].split('/')[-1])

        data_image,hdr = fits.getdata(imglist[imgnum],header=True)
        Times[filtername][imgnum] = hdr['MJD-OBS']
        
        # do the photometry!
        phot_table = aperture_photometry(data_image, apers)

        flux0 = np.array(phot_table['aperture_sum_0']) - \
        (area_of_ap/area_of_background)*np.array(phot_table['aperture_sum_1'])

        #
        
        # stuff into a new matrix
        #
        Photometry[filtername][imgnum] = flux0
        
        #
        # error stuff
        #
        skyvar = np.std(data_image[xboxcorner:xboxcorner+boxsize,yboxcorner:yboxcorner+boxsize])**2.
    
        # want to check and make sure this is actually background (percentile check?)
    
        err1 = area_of_ap * skyvar  # scatter in sky values
    
        err2 = flux0/GAIN # Poisson error
    
        err3 = skyvar*(area_of_ap)**2./(boxsize*boxsize) # uncertainty in mean sky brightness
    
        #print ('Scatter in sky values: ',err1**0.5,', uncertainty in mean sky brightness: ',err3**0.5)
    
        errtot = (err1 + err2 + err3)**0.5
        
        #now we need error handling...
        ePhotometry[filtername][imgnum] = errtot
        
        # should likely also do cleaning here, to nan out negative values
        Photometry[filtername][imgnum][Photometry[filtername][imgnum] <= 0.] = np.nan
        
        
        


In [ ]:
# detrend all stars


cPhotometry = {}

for filtername in filters:

    nimages = len(Photometry[filtername][:,0])
    imgindex = np.arange(0,nimages,1)
    cPhotometry[filtername] = np.zeros_like(Photometry[filtername])


    # get median flux value for each star (find percent change)
    for star in range(0,nstars):
        # apply an SN cut?
        SNval = Photometry[filtername][:,star]/ePhotometry[filtername][:,star]
        
        low_sn = np.where(SNval < 3.)
        # blank out bad photometry
        #print(len(low_sn[0]))
        Photometry[filtername][low_sn,star] = np.nan
        
        # now find the median
        med_val = np.nanmedian(Photometry[filtername][:,star]) 
        
        if med_val <= 0.0: # known bad photometry
            cPhotometry[filtername][:,star] = Photometry[filtername][:,star]*np.nan
        else:
            cPhotometry[filtername][:,star] = Photometry[filtername][:,star]/med_val
            
        # do a check for outlier photometry?
        


    # remove large-scale image-to-image variation to find best stars
    for night in range(0,nimages):
        cPhotometry[filtername][night,:] = cPhotometry[filtername][night,:]/np.nanmedian(cPhotometry[filtername][night])

    # eliminate stars with outliers from consideration
    for star in range(0,nstars):
        w = np.where( (cPhotometry[filtername][:,star] < 0.5) | (cPhotometry[filtername][:,star] > 1.5))
        cPhotometry[filtername][w,star] = np.nan
        

for filtername in filters:
    
    plt.figure()
    # check how we did via plot
    for star in range(0,100):
        plt.scatter(Times[filtername]-np.nanmin(Times[filtername]),cPhotometry[filtername][:,star],s=1.,color='black')



    plt.ylim(-1.,4.)
    plt.xlim(0,np.max(Times[filtername]-np.nanmin(Times[filtername])))

    plt.ylabel('Observation Time [days]')
    plt.xlabel('Image Number')
    plt.title(filtername)

In [ ]:


for filtername in filters:
    plt.figure()
    # check how individual nights did
    night_err = np.nanstd(cPhotometry[filtername],axis=1)
    plt.scatter(Times[filtername]-np.nanmin(Times[filtername]),night_err,color='black',s=3.)
    #plt.ylim(0.,0.5)

    plt.ylabel('$\sigma$',size=24)
    plt.xlabel('Time',size=24)
    plt.title(filtername,size=24)

# might think about rejecting images that are very uncertain

In [ ]:
# select the stars with which to do the comparison

filtername='R'

star_err = np.nanstd(cPhotometry[filtername],axis=0)

accuracy_threshold = 0.025

# we may also want a flux cut?
most_accurate = np.where( (star_err < accuracy_threshold) & (star_err > 0.) & (np.nanmedian(Photometry[filtername],axis=0) < 1000.) & (np.nanmedian(Photometry[filtername],axis=0) > 100.))[0]

print(len(most_accurate),':',most_accurate[0:5])
#plt.plot(star_err[star_err.argsort()])

# select stars based on the R band similar brightness?

# choose a representative star from different magnitude bands.

plt.figure()

print(np.log10(np.nanmedian(Photometry[filtername],axis=0))[most_accurate],np.log10(star_err)[most_accurate])



for filtername in filters:
    plt.figure()
    plt.scatter(np.log10(np.nanmedian(Photometry[filtername],axis=0)),np.log10(star_err),color='black',s=1.)
    plt.scatter(np.log10(np.nanmedian(Photometry[filtername],axis=0))[most_accurate],np.log10(star_err)[most_accurate],color='red',s=10.)
    plt.xlabel('log Flux',size=24)
    plt.ylabel('log Uncertainty',size=24)
    plt.title(filtername,size=24)
    plt.xlim(-1.,6)

In [ ]:

for filtername in filters:

    plt.figure()
    for num in most_accurate:

        plt.scatter(Times[filtername]-np.nanmin(Times[filtername]),cPhotometry[filtername][:,num],color='black',s=1.)


    plt.ylabel('Detrended Flux',size=24)
    plt.xlabel('Time',size=24)
    plt.title(filtername,size=24)

    plt.ylim(0.8,1.2)

## Differential Photometry!

Now with the best stars for non-variability identified, perform the actual differential photometry.

In [ ]:
# calculate each star's median value


dPhotometry = {}

# carried error from photometry
edPhotometry = {}

# error on differential photometry alone
eedPhotometry = {}

# total error
tePhotometry = {}


for filtername in filters:
    print(filtername)
    


    
    nimages = len(Photometry[filtername][:,0])
    
    imgindex = np.arange(0,nimages,1)



    dPhotometry[filtername] = np.zeros_like(Photometry[filtername])
    edPhotometry[filtername] = np.zeros_like(Photometry[filtername])
    eedPhotometry[filtername] = np.zeros_like(Photometry[filtername])
    tePhotometry[filtername] = np.zeros_like(Photometry[filtername])


    for star in range(0,nstars):

        tmp_phot = np.zeros([nimages,len(most_accurate)])

        for compindex,comp in enumerate(most_accurate):
            tmp_phot[:,compindex] = (Photometry[filtername][:,star]*np.nanmedian(Photometry[filtername][:,comp]))\
                                /(Photometry[filtername][:,comp]*np.nanmedian(Photometry[filtername][:,star]))

        dPhotometry[filtername][:,star] = np.nanmedian(tmp_phot,axis=1)

        # apply final scaling factors to the photometric error
        edPhotometry[filtername][:,star] = ePhotometry[filtername][:,star]*(np.nanmedian(tmp_phot,axis=1)/Photometry[filtername][:,star])

        # the differential photometry error
        eedPhotometry[filtername][:,star] = np.nanstd(tmp_phot,axis=1)

        # the differential photometry error
        tePhotometry[filtername][:,star] = ((ePhotometry[filtername][:,star]*(np.nanmedian(tmp_phot,axis=1)/Photometry[filtername][:,star]))**2. +\
                                            (np.nanstd(tmp_phot,axis=1))**2.)**0.5



## Run photometry on individual stars and make a plot with variability

In [ ]:

star = 860

print(xpos[star],ypos[star])


f = open(source+'of_interest_sources.reg','w')
f.write('circle '+str(xpos[star])+' '+str(ypos[star])+' '+str(int(aprad))+' # width=4 color=red\n') # or whatever you want the radius to be!
f.write('circle '+str(xpos[star])+' '+str(ypos[star])+' '+str(int(aprad+skybuff))+' # width=4 color=blue\n') # or whatever you want the radius to be!
f.write('circle '+str(xpos[star])+' '+str(ypos[star])+' '+str(int(aprad+skybuff+skywidth))+' # width=4 color=blue\n') # or whatever you want the radius to be!



f.close()



for fnum,filtername in enumerate(filters):
    

    TT = Times[filtername]-np.nanmin(Times[filtername])
    DD = dPhotometry[filtername][:,star]
    ED = tePhotometry[filtername][:,star]

    w = np.where(np.isfinite(TT) & np.isfinite(DD) & np.isfinite(ED))
    
    print(len(w[0]))

    ls = LombScargle(TT[w],DD[w],ED[w])
    
    try:
        frequency, power = ls.autopower()
    except:
        print('failed in {}'.format(filtername))
        continue
        
    fig = plt.figure(fnum,figsize=(8,3))

    ax0 = fig.add_axes([0.2,0.2,0.22,0.6])
    ax1 = fig.add_axes([0.48,0.2,0.22,0.6])
    ax2 = fig.add_axes([0.76,0.2,0.22,0.6])




    nimages = len(DD)




    ax0.scatter(TT,DD,color='black',s=10.)

    for num in range(0,nimages):
        ax0.plot([TT[num],TT[num]],\
                 [DD[num]-ED[num],DD[num]+ED[num]],\
                 color='black',lw=0.75,zorder=-9)


    ax0.set_xlabel('Time [days]',size=20)
    ax1.set_xlabel('Period [days]',size=20)
    ax0.set_ylabel(filtername,size=20)
    #ax0.set_xlim(0.,3.)

    ax1.plot(1./frequency,power,color='black')
    ax1.set_xlim(0.,10.)
    ax1.set_ylim(0.,1.)
    
    w = np.where( (1./frequency > 0.2) & (1./frequency < 4.))

    best_frequency = frequency[w][np.argmax(power[w])]

    print(1./best_frequency)
    newtime = TT % (1./best_frequency)

    #print(newtime)
    newtime = newtime - np.round(newtime,0) + 0.5

           
    ax2.scatter((newtime)-1.0,DD,color='gray',s=5.)
    ax2.scatter((newtime)+1.0,DD,color='gray',s=5.)


    for num in range(0,nimages):
        ax2.plot([newtime[num]-1.,newtime[num]-1.],\
             [dPhotometry[filtername][num,star]-edPhotometry[filtername][num,star],dPhotometry[filtername][num,star]+edPhotometry[filtername][num,star]],\
             color='gray',lw=0.5,zorder=-9)
        ax2.plot([newtime[num]+1.,newtime[num]+1.],\
             [dPhotometry[filtername][num,star]-edPhotometry[filtername][num,star],dPhotometry[filtername][num,star]+edPhotometry[filtername][num,star]],\
             color='gray',lw=0.5,zorder=-9)
        ax2.plot([newtime[num],newtime[num]],\
             [dPhotometry[filtername][num,star]-edPhotometry[filtername][num,star],dPhotometry[filtername][num,star]+edPhotometry[filtername][num,star]],\
             color='black',lw=0.75,zorder=-9)




    ax2.scatter((newtime),DD,color='black',s=15.)
    


    ax2.plot([-.5,1.5],[1.,1.],color='red',linestyle='dashed')
    ax2.set_xlabel('Phase',size=20)
    #ax2.set_ylabel('Relative Photometry',size=24)
    ax2.set_xlim(-0.5,1.5)

In [ ]:
limits = np.arange(1,8.6,0.5)
print(limits)

period_days = 1./frequency

for value in range(1,len(limits)):
    w = np.where( (period_days > limits[value-1]) & (period_days < limits[value]))
    
    try:
        best_period = period_days[w][np.argmax(power[w])]
        best_power  = power[w][np.argmax(power[w])]

        print(best_period,best_power)
        
    except:
        print('No Maximum for {}-{}'.format(limits[value-1],limits[value]))

